Project 1 Part 3 Samuel Andrews

In [177]:
import re

#read in the file

with open('project_1_part_2_output.txt') as f:
    contentlines = f.readlines()
contentlines[0:2]

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS /17  /17  256945       00 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO\n',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n']

While the data in its current state is now all wrapped up into individual lines, we unfortunatley have a new issue. From the starting document, 17 varibles are listed. However, based on the output there are 22 data items. 2 of those item appear to be the year portion of the two dates listed off (as shown in the top portion of the triage discussion), but the remaining three take the forms "AABBC", "1706149900003#STA=CERS", and "854300" in most cases, with the latter somtimes not being present at all. Due to how the dates ended up being spread across multiple lines, we can infer that some og these trailing items are simply parts of variables in the wrong place, which is best scene when viewing the data in its original form. It seems that every other line represents a "cut-out" of the line above it, allows us to infer that the the floating /17's belong on the ends of the msuhed together dates, the standalone 2 digits are the seconds in the "Hour:Minutes:Seconds" and that the first set of random digits is likely the tail end of the digits smushed against the data. That leaves two remaining items, one of which is optional.

Logically, they don't appear to fit anywhere, and without metadata I have no way to know for sure, so to avoid the risk of unintentionally destorying the data, I will place them in seperate columns with the understanding that in a "real-world" case there would be someone I could talk to who would know where which column these items belong two and one could easily pair them up using simple list comprehensions to put in its ture form after gleaning such.

Given what was discussed above and the nature of our overall approach, the first goal will to be to put every line in a similar enough form to pull out the data items with the same logic. Starting off, those random "/17" are always the same, so lets remove them outright so they don't have to be dealt with.

First, we need a function to replace such transgressions with empty space

In [178]:
#get testline

testline = contentlines[1]
testline

'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n'

In [179]:
#test function

remove_17 = lambda line: line.replace(" /17 ", "")

remove_17(testline)

'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX 866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n'

In [180]:
#for all lines

testoupout = [remove_17(line) for line in contentlines ]

In [181]:
#look at it

testoupout[:10]

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS 256945       00 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO\n',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX 866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX 866096       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX 869780       23 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300\n',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX 892888       29 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA

In [182]:
#create new variable to hold our work as we go through

Workingcontent = testoupout

Next, to keep future regular expressions clean, lets make sure every variable is only seperated by a singular whitespace. A cheeky way of doing this in python is to take advantage of str.split seperating on excess whitespace and then using str.join to rejoin the characters with only a single space.

In [183]:
#split by whiteapce, then recombine seperated by spaces

Excess_space_remover = lambda line: " ".join(line.split())

Excess_space_remover(testline)

'CERS SS NEW 100 2.36000018422.360000 2.37000006/1406/191706149900003 10:20:ContraSSFREX /17 /17 866087 04 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300'

In [184]:
#do it for all lines

testoupout = [Excess_space_remover(line) for line in Workingcontent]

testoupout[0:10]

['CERS SS NEW 2,756 2.400000 2.340000 2.45000006/1406/191706149900003 09:30:CustSS 256945 00 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=GSCO',
 'CERS SS NEW 100 2.36000018422.360000 2.37000006/1406/191706149900003 10:20:ContraSSFREX 866087 04 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 200 2.350000 2.360000 2.37000006/1406/191706149900003 10:20:ContraSSFREX 866096 04 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 100 2.350000 2.350000 2.36000006/1406/191706149900003 10:20:ContraSSFREX 869780 23 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 100 2.350000 2.350000 2.36000006/1406/191706149900003 10:22:ContraSSFREX 892888 29 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 100 2.360000 2.350000 2.36000006/1406/191706149900003 10:24:ContraSSFREX 909119 01 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS',
 'CERS SS NEW 100 2.36000085432.350000 2.36

In [185]:
#turn into working content

Workingcontent = testoupout

Because the end result of this project is a clean .csv file, the commas in this file will need to be expunged. Fortunatley,they only ever appear sometimes in the fourth data item, which is presumed to be qty where they cause damage to the numeric data anyway. So while we have the data so neatly spaced, we might as well remove them now similar to how we remove the "/17s" a step earlier.

In [186]:
#showcase the problem

testline = testoupout[0]

testline


'CERS SS NEW 2,756 2.400000 2.340000 2.45000006/1406/191706149900003 09:30:CustSS 256945 00 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=GSCO'

In [187]:
#function to fix it

remove_comma = lambda line: line.replace(",", "")

remove_comma(testline)

'CERS SS NEW 2756 2.400000 2.340000 2.45000006/1406/191706149900003 09:30:CustSS 256945 00 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=GSCO'

In [188]:
#for all lines

testoupout = [remove_comma(line) for line in Workingcontent ]

testoupout[0:10]


['CERS SS NEW 2756 2.400000 2.340000 2.45000006/1406/191706149900003 09:30:CustSS 256945 00 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=GSCO',
 'CERS SS NEW 100 2.36000018422.360000 2.37000006/1406/191706149900003 10:20:ContraSSFREX 866087 04 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 200 2.350000 2.360000 2.37000006/1406/191706149900003 10:20:ContraSSFREX 866096 04 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 100 2.350000 2.350000 2.36000006/1406/191706149900003 10:20:ContraSSFREX 869780 23 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 100 2.350000 2.350000 2.36000006/1406/191706149900003 10:22:ContraSSFREX 892888 29 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300',
 'CERS SS NEW 100 2.360000 2.350000 2.36000006/1406/191706149900003 10:24:ContraSSFREX 909119 01 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS',
 'CERS SS NEW 100 2.36000085432.350000 2.360

In [189]:
Workingcontent = testoupout

Before we get ahead of ourselves and begin pulling data, we need to talk about a particular error that some lines suffer from. Take line 79 for example and compare it to line 2:

In [190]:

print("Wrong")
print(Workingcontent[78])

print("\n\n")

print("\"Correct\"")
print(Workingcontent[1])


Wrong
CERS SS NEW 79 2.350000 2.350000 2.36000006/1406/191706149900004 10:56:ContraSSFREX 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 271819 34 854300



"Correct"
CERS SS NEW 100 2.36000018422.360000 2.37000006/1406/191706149900003 10:20:ContraSSFREX 866087 04 0704 PerUnit TERM 0.010000 AABBC 1706149900003#STA=CERS 854300


In some cases, we can see that the hanging "seconds" item and the miscellanues digit group in front of it are sometimes places just before the end instead of after the time variable. This is not a problem that is insurmountable, but it is very annoying and could lead to some fairly disgusting regex statements. This could be fixed by reorder each line by first determing problem lines via regex and then using indexing to fix it, but because we have yet to fix the variable bleed problem, that solution would be incredibly finicky to pull off. Instead, we will opt to avoid the problem entirely by grabbing off everything up until the discrepancy in order takes place, and than writing the regex statement to deal specifically with that group.

So to start, well work item by item until we get to that point to ensure that our pulls are without error until we get to that point. By the end of this list, we should have a bunch of lists sized 913 that we can join together into our clean .csv file.

First up, lets grab the symbol using the regex developed for part 2, as we know the ticker.

In [191]:

#test regex

pullticker = re.compile('^([A-Z][A-Z]?[A-Z]?[A-Z]?)\s')

pullticker.match(Workingcontent[1])[1]



'CERS'

In [192]:
#for all lines


Ticker_list = [pullticker.match(line)[1] for line in Workingcontent]

Ticker_list[0:20]




['CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS']

In [193]:
#check length

len(Ticker_list)

913

The next variable, side, is literally just SS 913 times (as shown by a quick control F, where you get 913*2 times across 2 variables). We could just make a list that size containing only that, but for possbile future use we will build a regex that grabs it all the same.

In [194]:
pullside = re.compile('\s(SS)\s')


pullside.search(Workingcontent[1])[1] #change to search cause were not in the middle

'SS'

In [195]:

side_list = [pullside.search(line)[1] for line in Workingcontent]

side_list[0:20]

['SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS']

In [196]:
len(side_list)

913

Next up is cxl, which is always new across the board just like side and will be treated the same.

In [197]:
pullcxl = re.compile('\s(NEW)\s')


pullcxl.search(Workingcontent[1])[1] 

'NEW'

In [198]:
cxl_list = [pullcxl.search(line)[1] for line in Workingcontent]

cxl_list[0:20]

['NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW',
 'NEW']

In [199]:
len(cxl_list)

913

Next, we have qty, which presents are first real challenge of note. This variable is a number amount seperated by spaces that comes directly after new. It used to contain commas, but those were removed earlier, so we can use new as a base to capture the value off of.

In [200]:
pullqty = re.compile('\sNEW\s(\S*)\s')


pullqty.search(Workingcontent[1])[1] 

'100'

In [201]:
qty_list = [pullqty.search(line)[1] for line in Workingcontent]

qty_list[0:20]

['2756',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '200',
 '100',
 '300',
 '100',
 '100']

In [202]:
len(qty_list)

913

Next, we have price, which somtimes bleeds into the next variable bid, which in turn sometimes bleeds into ask. Ask also bleeds into the next date variable, whihc always starts with a 06 as discussed in Triage. there is also another 06 later in the lines that is also accounted for. It appears the bleeding characters come from an odd rounding error caused by these prices being stored as floats. However, these floats always start with "x.", where x is any integer, which always starts each of these numbers off. Homing in on new again, we can use this trick to grab all three in one fell swoop.

In [203]:
pull_price_bid_ask = re.compile('\sNEW\s\S*\s(\d\.\S*)\s?(\d\.\S*)\s?(\d\.\S+)\s?06\/\S+06\/')

pull_price_bid_ask.search(Workingcontent[1])[1] 

'2.3600001842'

In [204]:

price_list = [pull_price_bid_ask.search(line)[1] for line in Workingcontent]

bid_list = [pull_price_bid_ask.search(line)[2] for line in Workingcontent]

ask_list = [pull_price_bid_ask.search(line)[3] for line in Workingcontent]



In [205]:
print(price_list[0:20])

print(bid_list[0:20])

print(ask_list[0:20])

['2.400000', '2.3600001842', '2.350000', '2.350000', '2.350000', '2.360000', '2.3600008543', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.365000', '2.3600008543', '2.360000', '2.360000', '2.360000', '2.360000', '2.365000']
['2.340000', '2.360000', '2.360000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.350000', '2.360000']
['2.450000', '2.370000', '2.370000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.360000', '2.370000']


In [206]:
print(len(price_list))

print(len(bid_list))

print(len(ask_list))

913
913
913


The next two variables, T-Dats-DatTRadeID and TradeTiSS, are always squished together and were already found indirectly by our last solution. We will grab both of them at the same time just the same. Afterwards, we will need to join "/17" on the end of each of these list items get them fully cleaned

In [207]:

pull_T_DatS_DatTradeID_TradeTiSS = re.compile('\sNEW\s\S*\s\d\.\S*\s?\d\.\S*\s?\d\.\S+\s?(06\/\S+)(06\/\d\d)')


pull_T_DatS_DatTradeID_TradeTiSS.search(Workingcontent[1])[1] 

'06/14'

In [208]:

T_DatS_DatTradeID_list = [pull_T_DatS_DatTradeID_TradeTiSS.search(line)[1] for line in Workingcontent]

TradeTiSS_list = [pull_T_DatS_DatTradeID_TradeTiSS.search(line)[2] for line in Workingcontent]

In [209]:
print(T_DatS_DatTradeID_list[0:10])

print(TradeTiSS_list[0:10])

['06/14', '06/14', '06/14', '06/14', '06/14', '06/14', '06/14', '06/14', '06/14', '06/14']
['06/19', '06/19', '06/19', '06/19', '06/19', '06/19', '06/19', '06/19', '06/19', '06/19']


In [210]:
print(len(T_DatS_DatTradeID_list))

print(len(TradeTiSS_list))

913
913


In [211]:
#add the "/17" back onto the ends of them

T_DatS_DatTradeID_list_fixed = [line + "/17" for line in T_DatS_DatTradeID_list]

TradeTiSS_list_fixed = [line + "/17" for line in TradeTiSS_list]

print(T_DatS_DatTradeID_list_fixed[0:10])

print(TradeTiSS_list_fixed[0:10])

['06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17', '06/14/17']
['06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17', '06/19/17']


The next variable, exbr, appears to be some kind of code that always bleeds into the date. However, a 6 digit piece of it appears to have been pulled away from it much like the "/17" and the seconds portion of the time. Due to the error desribed earlier, it also is does not consistently appear in the same place, and another optional code might conflict with it. The good news is that it does always come first, so with some regex logic we can still scoop both. 

In [212]:
pull_exbr = re.compile('\/\d\d(\d+)\s.*\s([0-9]{6})\s.*\s([0-9]{6})?\s?.*?')


print(pull_exbr.search(Workingcontent[1])[1])

print(pull_exbr.search(Workingcontent[1])[2])

1706149900003
866087


In [213]:
exbr_part1_list = [pull_exbr.search(line)[1] for line in Workingcontent]

exbr_part2_list = [pull_exbr.search(line)[2] for line in Workingcontent]



In [214]:
print(exbr_part1_list[0:10])

print(exbr_part2_list[0:10])

['1706149900003', '1706149900003', '1706149900003', '1706149900003', '1706149900003', '1706149900003', '1706149900003', '1706149900003', '1706149900003', '1706149900003']
['256945', '866087', '866096', '869780', '892888', '909119', '909128', '909140', '909143', '909148']


In [215]:
print(len(exbr_part1_list))
print(len(exbr_part2_list))

913
913


In [216]:
#now, to combine the list we will use a list comprehension and the zip function

exbr_complete = [part1 + part2 for part1, part2 in zip(exbr_part1_list, exbr_part2_list)] 

exbr_complete[0:10]

['1706149900003256945',
 '1706149900003866087',
 '1706149900003866096',
 '1706149900003869780',
 '1706149900003892888',
 '1706149900003909119',
 '1706149900003909128',
 '1706149900003909140',
 '1706149900003909143',
 '1706149900003909148']

In [217]:
#check length once last time

len(exbr_complete)

913

Past this point, without metadata, I lack the full confidence to place any variable into a specific column. However, every element is still uniquely identifiable, so despite not knowing what they represent, they still can be isolated and made into a clean .csv file as discussed at the begining of this notebook.

The next variable is the hour:minute:second variable spoken of prior. It similarly is spread into 2 parts and can therefore be handled in roughly the same manner.



In [218]:
pull_time = re.compile('\s(\d\d:\d\d:).*\s(\d\d)\s')


print(pull_time.search(Workingcontent[1])[1])

print(pull_time.search(Workingcontent[1])[2])

10:20:
04


In [219]:
time_part1_list = [pull_time.search(line)[1] for line in Workingcontent]

time_part2_list = [pull_time.search(line)[2] for line in Workingcontent]

In [220]:
print(time_part1_list[0:10])

print(time_part2_list[0:10])

['09:30:', '10:20:', '10:20:', '10:20:', '10:22:', '10:24:', '10:24:', '10:24:', '10:24:', '10:24:']
['00', '04', '04', '23', '29', '01', '01', '01', '01', '01']


In [221]:
print(len(time_part1_list))
print(len(time_part2_list))

913
913


In [222]:
#combine again

time_complete = [part1 + part2 for part1, part2 in zip(time_part1_list, time_part2_list)] 

time_complete[0:10]

['09:30:00',
 '10:20:04',
 '10:20:04',
 '10:20:23',
 '10:22:29',
 '10:24:01',
 '10:24:01',
 '10:24:01',
 '10:24:01',
 '10:24:01']

In [223]:
print(len(time_complete))

913


Next, we have what I believe is the TradeCommtype. It makes up the remaining characters that come right after the first part of the time variable we just pulled out, meaning we need only to slightly modify our current logic to grab it.

In [224]:
pull_TradeCommtype = re.compile('\s\d\d:\d\d:(\S+)')


print(pull_TradeCommtype.search(Workingcontent[1])[1])

ContraSSFREX


In [225]:
TradeCommtype_list = [pull_TradeCommtype.search(line)[1] for line in Workingcontent]

TradeCommtype_list[0:10]

['CustSS',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX',
 'ContraSSFREX']

In [226]:
len(TradeCommtype_list)

913

The next variable Is what I presume to be the account number, as it is usually found in close relation to that category and remains consistant throughout the entire pull. But again, for logic puproses, we still will right a simple regex to grab it.

In [227]:
pull_accountid = re.compile('\s(\d\d\d\d)\s')


print(pull_accountid.search(Workingcontent[1])[1])

0704


In [228]:
accountid_list = [pull_accountid.search(line)[1] for line in Workingcontent]

accountid_list[0:10]

['2756',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704']

In [229]:
len(accountid_list)

913

The next variable is always perunit, but I am not sure which variable it belongs under. Regardless, it will be pulled in exactly the same matter, only more discreetly to avoid certain difficulty.

In [230]:
pull_PerUnit = re.compile('\s(PerUnit)\s')


print(pull_PerUnit.search(Workingcontent[1])[1])

PerUnit


In [231]:
perUnit_list = [pull_PerUnit.search(line)[1] for line in Workingcontent]

perUnit_list[0:10]

['PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit']

In [232]:
len(perUnit_list)

913

Term is in the exact same boat as PerUnit, the previous solution is used again but simplified further to keep things fresh.

In [233]:
pull_TERM = re.compile('\s(TERM)\s')

Term_list = [pull_TERM.search(line)[1] for line in Workingcontent]

len(Term_list)


913

As we begin to finish up, we come across a variable that appears to be a floating point variable isolated by itself that comes at some point after the time variable of the form (0.xxxxxx). Though the implementation gets a bit stale, we remind ourselves that we want the data in this multi-list format incase we get the variable wrong so that they may be swapped easily.

In [234]:
pull_float = re.compile('\s(0\.[\d]{6})\s')


print(pull_float.search(Workingcontent[1])[1])

0.010000


In [235]:
float_list = [pull_float.search(line)[1] for line in Workingcontent]

float_list[0:10]

['0.010000',
 '0.010000',
 '0.010000',
 '0.010000',
 '0.010000',
 '0.010000',
 '0.010000',
 '0.010000',
 '0.010000',
 '0.010000']

In [236]:
len(float_list)

913

As we come into the last three items, we are met with a variable that conists of simply 5 to 6 capital letters. The regex once again follows the same basic pattern.

In [237]:
pull_Charvar = re.compile('\s([A-Z]{5,6})\s')


print(pull_Charvar.search(Workingcontent[1])[1])

AABBC


In [238]:
Charvar_list = [pull_Charvar.search(line)[1] for line in Workingcontent]

Charvar_list[0:10]

['AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC']

In [239]:
len(Charvar_list)

913

The penultimate data item (ex. 1706149900003#STA=GSCO) is fairly straitforward to grab, as it is the only item to always have an octothorpe (#) in its name. However, its exact nature is perplexing, as every portion of the value seems listed elsewhere, as it seems to be a combination of the first part of what I have called Exbkr, STA=, and then a Ticker Symbol which may or may not match the one already in place. Despite this, data is data, and it will be pulled and isolated as such. It may or may not be on the end of a line as well, so that must be accounted for.

It may also be more then one variable, but again, without metadata, I have no way to confidently make a distinction.

In [240]:
pull_Unknown_Octo = re.compile('\s(\S+#\S+)\s?')

print(pull_Unknown_Octo.search(Workingcontent[1])[1])

1706149900003#STA=CERS


In [241]:
Unknown_Octo_list = [pull_Unknown_Octo.search(line)[1] for line in Workingcontent]

Unknown_Octo_list[0:10]

['1706149900003#STA=GSCO',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS',
 '1706149900003#STA=CERS']

In [242]:
len(Unknown_Octo_list)

913

The last variable to be grabbed rests on the end, and is an optional 6 digit code similar to part 2 of Exbkr. However, because it only exists optionally, we will have to handle the case where it doesn't show up, so we will need to construct a more complex lambda statement. As for the regex, this code is quick CRTL+F and some minor searching tells us that the code is always either 854300 or 473640, which gives us a pretty discrete solution as far as regex is concerned. It was also ensured that these numbers never appeared anywhere else, so we don't have to worry about false positives.

After carefeul consideration, it was decided that these numerica digits were the trailing digits of what is currently called the "unknown data" similar to how T-Dats-DatTRadeID had disjointed codes. Furthermore, everything after the # in that unknown portion will be considered as its own unique bit of data and given its own column.

In [243]:
pull_Optional_Code = re.compile('(473640|854300)')


#should fail
print(bool(pull_Optional_Code.search(Workingcontent[0])))

#should succeed
print(bool(pull_Optional_Code.search(Workingcontent[1])))

False
True


In [244]:


Optional_end_func = lambda line: (pull_Optional_Code.search(line)[1] if bool(pull_Optional_Code.search(line)) == True else "N/A")


In [245]:

Unknown_End_Code = [Optional_end_func(line) for line in Workingcontent]

Unknown_End_Code[0:10]


['N/A',
 '854300',
 '854300',
 '854300',
 '854300',
 'N/A',
 '854300',
 '854300',
 '854300',
 '854300']

In [246]:
len(Unknown_End_Code)

913

Now, we need to split up the unkowndata into two columns. 

In [247]:
#test on single case
testrow = Unknown_Octo_list[0]

print(testrow)

print(testrow.partition("#")[0])


1706149900003#STA=GSCO
1706149900003


In [248]:
#do it for all elements

Unknown_ID = [line.partition("#")[0] for line in Unknown_Octo_list]

Unknown_ID[0:20]




['1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003',
 '1706149900003']

In [249]:
#combine with End code.

Full_Unknown_Code = [frontcode + backcode for frontcode, backcode in zip(Unknown_ID, Unknown_End_Code)]

Full_Unknown_Code[0:20]


['1706149900003N/A',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003N/A',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003N/A',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300']

This works, but now we have a problem with certain endcodes being missing. Given the pattern, we can surmise what the missing values should be based on the ones surronding it. If it ends in "3N/A", it should be 854300, if it ends with "4N/A" it should be 473640



In [250]:
#function to replace

Code_fixer = lambda line: line.replace("N/A" , "854300") if "3N/A" in line else line.replace("N/A", "473640")


#for all elements

Full_Unknown_Code = [Code_fixer(line) for line in Full_Unknown_Code]

Full_Unknown_Code[0:20]


['1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300',
 '1706149900003854300']

Now, we just need to grab the second variable from the original uknown code.

In [251]:
#use the same partition method


Unknown_variable_pound = [line.partition("#")[1] + line.partition("#")[2] for line in Unknown_Octo_list]

Unknown_variable_pound[0:20]

['#STA=GSCO',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS',
 '#STA=CERS']

At this point, we have all of the data items pulled out, now it is just a matter of putting them back together in a way that makes sense. Again, as stated above, I do not feel confident in matching certain columns to certain data types, so I tried my best guess and ended up with more columns then were indicated at the start, but if any adjustments are needed it is as simple as recombining the pulled data items to suit whatever the need it.

First, we must make the function to combine the lists

In [252]:
list_merger = lambda list1, list2: [part1 + ","+ part2 for part1, part2 in zip(list1, list2)] 

In [253]:
#build the list however you want.

WorkingOutput = list_merger(Ticker_list,  side_list)

WorkingOutput = list_merger(WorkingOutput,  cxl_list)

WorkingOutput = list_merger(WorkingOutput,  qty_list)

WorkingOutput = list_merger(WorkingOutput,  price_list)

WorkingOutput = list_merger(WorkingOutput,  bid_list)

WorkingOutput = list_merger(WorkingOutput,  ask_list)

WorkingOutput = list_merger(WorkingOutput,  T_DatS_DatTradeID_list_fixed)

WorkingOutput = list_merger(WorkingOutput,  TradeTiSS_list_fixed)

WorkingOutput = list_merger(WorkingOutput, exbr_complete)

WorkingOutput = list_merger(WorkingOutput,  time_complete)

WorkingOutput = list_merger(WorkingOutput,  TradeCommtype_list)

WorkingOutput = list_merger(WorkingOutput,  accountid_list)

WorkingOutput = list_merger(WorkingOutput,  perUnit_list)

WorkingOutput = list_merger(WorkingOutput,  Term_list)

WorkingOutput = list_merger(WorkingOutput,  float_list)

WorkingOutput = list_merger(WorkingOutput,  Charvar_list)

WorkingOutput = list_merger(WorkingOutput,  Full_Unknown_Code)

WorkingOutput = list_merger(WorkingOutput,  Unknown_variable_pound)


WorkingOutput[0:20]

['CERS,SS,NEW,2756,2.400000,2.340000,2.450000,06/14/17,06/19/17,1706149900003256945,09:30:00,CustSS,2756,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=GSCO',
 'CERS,SS,NEW,100,2.3600001842,2.360000,2.370000,06/14/17,06/19/17,1706149900003866087,10:20:04,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,200,2.350000,2.360000,2.370000,06/14/17,06/19/17,1706149900003866096,10:20:04,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.360000,06/14/17,06/19/17,1706149900003869780,10:20:23,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.360000,06/14/17,06/19/17,1706149900003892888,10:22:29,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,100,2.360000,2.350000,2.360000,06/14/17,06/19/17,1706149900003909119,10:24:01,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003

Now, the columns need tobe named. Without metadata or an example output, columns names were taken from a combination of the original names and new names the describe the variable to the best of my understanding. The former were used whenever possible, but due to my suspicion that the data was originally output with certain columns violated the golden rule even when in a clean state, some were parititoned or thrown out.

Even as they stand though, the column names do not appear to make a whole lot of sense. However, without the context, perhaps they never will. Whats more important is that the data items are isolated, the names at this point are trivial and can be changed on a whim if nescescary. 

In [254]:
#add add a row to act as columns

Columns = "Symbol,Side,CxL,Qty,Price,Bid,Ask,StartDate,EndDate,TradeID,Time,Exkbr,TradeCommType,SourceComission,AccountType,OrderID,GTL,TrailerInfo,Clr"

WorkingOutput.insert(0, Columns)

WorkingOutput[0:20]



['Symbol,Side,CxL,Qty,Price,Bid,Ask,StartDate,EndDate,TradeID,Time,Exkbr,TradeCommType,SourceComission,AccountType,OrderID,GTL,TrailerInfo,Clr',
 'CERS,SS,NEW,2756,2.400000,2.340000,2.450000,06/14/17,06/19/17,1706149900003256945,09:30:00,CustSS,2756,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=GSCO',
 'CERS,SS,NEW,100,2.3600001842,2.360000,2.370000,06/14/17,06/19/17,1706149900003866087,10:20:04,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,200,2.350000,2.360000,2.370000,06/14/17,06/19/17,1706149900003866096,10:20:04,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.360000,06/14/17,06/19/17,1706149900003869780,10:20:23,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS,SS,NEW,100,2.350000,2.350000,2.360000,06/14/17,06/19/17,1706149900003892888,10:22:29,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS',
 'CERS

In [255]:
#now, make it all one line

WorkingOutput_in_one_line = "\n".join(WorkingOutput)

WorkingOutput_in_one_line[0:500]

'Symbol,Side,CxL,Qty,Price,Bid,Ask,StartDate,EndDate,TradeID,Time,Exkbr,TradeCommType,SourceComission,AccountType,OrderID,GTL,TrailerInfo,Clr\nCERS,SS,NEW,2756,2.400000,2.340000,2.450000,06/14/17,06/19/17,1706149900003256945,09:30:00,CustSS,2756,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=GSCO\nCERS,SS,NEW,100,2.3600001842,2.360000,2.370000,06/14/17,06/19/17,1706149900003866087,10:20:04,ContraSSFREX,0704,PerUnit,TERM,0.010000,AABBC,1706149900003854300,#STA=CERS\nCERS,SS,NEW,200,2.350000,2.3'

In [256]:
#wite to .csv

with open('Final_output_adjusted2.csv','w') as outfile:
    outfile.write(WorkingOutput_in_one_line)